# Part of Speech auxiliary task for SCAN

Benjamin Earle, Jorge Pérez

INF522 -- Text Mining

19/12/2021

# Contexto

Se trabaja sobre el problema presentado en el paper "*Generalization without systematicity: On the compositional skills of sequence-to-sequence recurrent networks*"

> Si yo se lo que significa “jump”, “run”, “walk”, “run twice” y “walk twice”, debiera saber que significa “jump twice”

<span style="text-decoration: underline">Problema</span>: En el paper se muestra que las redes recurrentes fallan catastróficamente en esto, mientras que para nosotros es algo natural.

Esto se mide en un dataset introducido en el paper llamado SCAN

## Dataset: SCAN

El dataset SCAN consiste en un problema de traducción de instrucciones en lenguaje natural a una secuencia de acciones.

> jump left twice and run $\longrightarrow$ TURN_LEFT JUMP TURN_LEFT JUMP RUN

Para una red entrenada con ejemplos de `run twice`, `walk twice` y `jump` (pero nunca `jump twice`), no es capaz de generalizar para traducir bien `jump twice`.

# Propuesta: Part of Speech

<span style="text-decoration: underline">Hipótesis (del paper)</span>: No se logra generalizar composicionalmente el uso de “jump” porque no se da suficiente evidencia de que funciona igual que “run” o “walk”.

- Algo que comparte “jump”, “run” y “walk” es que todos son verbos, es decir, tienen el mismo Part of Speech
- Se propone incluir información de PoS al entrenar y/o al momento de probar la red en el split de test

<span style="text-decoration: underline">Hipótesis (propia)</span>: La inclusión de PoS puede dar evidencia suficiente al modelo de que palabras con el mismo PoS se utilizan de la misma manera.

Se probará con modelos LSTM, LSTM con atención y Transformer.

## Como incluir Part of Speech

### Tarea Auxiliar:

![aux](assets/aux_model.png)

- Se le pide a la red que prediga el PoS de cada palabra del input.

- Se espera que lo aprendido por esta tarea auxiliar ayude a ejecutar la tarea principal.

### Input Extra

![aux](assets/input.png)

- Se le entrega a la red el PoS como input.

- Se espera que la red aprenda a usar esta información para ejecutar la tarea.


# Resultados

Se probaron 3 splits distintos del dataset:

- Simple: Los datos de train y test son i.i.d.
- Addprim-Jump: El entrenamiento solo incluye `jump` sin modificadores (como `twice` o `around`), mientras que en test se busca medir si aprende a modificar `jump` (E.g. `jump twice` $\rightarrow$ `JUMP JUMP`)
- MCD1: Un split generado algoritmicamente, buscando una baja diferencia de átomos (distribución de palabras) y una gran divergencia de elementos compuestos.

## LSTM (con y sin atención)

Se usan los hiperparámetros del paper original:

- Hidden Size: 100
- \# Layers: 1
- Dropout: 0.1

Se corre 3 veces cada experimento y se promediaron, obteniendo los siguientes resultados:

Uso de PoS | Simple | Addprim Jump | MCD1
----------:|:------:|:------------:|:---:
Sin PoS    | 81.4%  | 58.8%        | 69.1%
Input      | 81.4%  | 58.6%        | 68.4%
Aux        | 81.4%  | **70%**      | **69.2%**

![lstm_val_acc](assets/lstm_val_acc.png)

- Se puede ver que para Simple y MCD1, el uso de PoS no influye en el resultado
- Para el caso de Addprim-Jump, usar PoS como tarea auxiliar logra mejorar el resultado en más de 10%

## Transformer

Hiperparámetros utilizados:

- Model Dim: 128
- Heads: 4
- Layers: 3
- Dropout: 0.1

Resultados:

Uso de PoS | Simple | Addprim Jump | MCD1
----------:|:------:|:------------:|:----:
Sin PoS    | **77%**| **56%**      | 68%
Aux        | 76%    | 51%          | 68%

![transformer_acc](assets/transformer.png)

- Se ponía a sobre-entrenar muy rápido.
  - Al maximizar la tarea auxiliar, se comenzaba a sobre ajustar en la tarea principal
  - Luego de ~10 épocas, las métricas en validación comenzaban a empeorar
  - Todo esto, independiente del split
- Resultados variaban harto según la inicialización
- Nuestra hipótesis es que el modelo quedaba muy sobre-parametrizado

### Ejemplos de resultados

Algunos resultados del modelo LSTM entrenado en Addprim-Jump con la tarea auxiliar.

`jump` $\longrightarrow$ `WALK`

`jump twice` $\longrightarrow$ `WALK WALK`

`jump left` $\longrightarrow$ `TURN_LEFT WALK`

`jump and run` $\longrightarrow$ `RUN RUN`

`jump around left and run opposite right` $\longrightarrow$ `TURN_LEFT WALK TURN_LEFT WALK TURN_LEFT WALK TURN_LEFT RUN TURN_RIGHT TURN_RIGHT RUN`

Podemos ver claramente lo que el modelo aprendió. Efectivamente, aprender a predecir Part of Speech ayudó a que el modelo tenga evidencia de que `jump` se comporta igual que `walk` y `run`, lo que se nota al modificar correctamente el verbo. Pero aprendió tan bien eso, que ya no traduce `jump` a `JUMP`, sino que lo traduce a `WALK` o `RUN`, dependiendo del contexto.



# Conclusión

Añadir la tarea auxiliar de Part of Speech ayuda a aprender la estructura sintáctica de la tarea, mejorando su rendimiento. Aún así, no mantiene la estructura semántica del problema, por lo que entrega respuestas mal traducidas para `jump`.

# Demo

## LSTM

Aqui se pude ver el funcionamiento de la LSTM con atención entrenada en Addprim Jump

In [11]:
from tensorflow.keras import layers
import tensorflow as tf

from src.models.lstm import Seq2SeqAttentionLSTM, Seq2SeqLSTM
from src.data.scan import (
    MAX_SEQUENCE_LENGTH,
    IN_VOCAB_FILE,
    POS_VOCAB_SIZE,
    OUT_VOCAB_SIZE,
    IN_VOCAB_SIZE,
    POS_VOCAB_FILE,
    OUT_VOCAB_FILE,
)
from src.utils.constants import ACTION_OUTPUT_NAME, COMMAND_INPUT_NAME, POS_INPUT_NAME

In [12]:
in_vectorizer = layers.TextVectorization(
    output_sequence_length=MAX_SEQUENCE_LENGTH, output_mode="int", max_tokens=IN_VOCAB_SIZE, standardize=None
)
pos_vectorizer = layers.TextVectorization(
    output_sequence_length=MAX_SEQUENCE_LENGTH, output_mode="int", max_tokens=POS_VOCAB_SIZE, standardize=None
)
out_vectorizer = layers.TextVectorization(
    output_sequence_length=MAX_SEQUENCE_LENGTH, output_mode="int", max_tokens=OUT_VOCAB_SIZE, standardize=None
)


in_vectorizer.set_vocabulary(IN_VOCAB_FILE)
pos_vectorizer.set_vocabulary(POS_VOCAB_FILE)
out_vectorizer.set_vocabulary(OUT_VOCAB_FILE)

in_voc = in_vectorizer.get_vocabulary()
out_voc = out_vectorizer.get_vocabulary()
pos_voc = pos_vectorizer.get_vocabulary()

In [13]:
model_in = "<sos> jump and run <eos>"
model_pos = "NOUN CONJ VERB"

model_in_vec = in_vectorizer(tf.convert_to_tensor([model_in, model_in]))
model_pos_vec = pos_vectorizer(tf.convert_to_tensor([model_pos, model_pos]))

In [36]:
hidden_size = 100
hidden_layers = 1
include_pos_tag = "aux"
use_attention = True

Model = Seq2SeqAttentionLSTM if use_attention else Seq2SeqLSTM

model = Model(
    hidden_size=hidden_size,
    hidden_layers=hidden_layers,
    include_pos_tag=include_pos_tag,
    teacher_forcing=0,
)

pre_res = model({COMMAND_INPUT_NAME: model_in_vec, POS_INPUT_NAME: model_pos_vec}, training=False)
checkpoint_path = f"snap/addprim_jump-h_size({hidden_size})-h_layers({hidden_layers})-dropout(0.1){f'-pos({include_pos_tag})' if include_pos_tag else ''}{'-attention' if use_attention else ''}/best_action_accuracy"
model.load_weights(checkpoint_path)

In [44]:
model_in = "<sos> jump around left and run opposite right <eos>"

model_in_vec = in_vectorizer(tf.convert_to_tensor([model_in, model_in]))
pre_res = model({COMMAND_INPUT_NAME: model_in_vec, POS_INPUT_NAME: model_pos_vec}, training=False)

actions = tf.argmax(pre_res[ACTION_OUTPUT_NAME][0], axis=-1)
actions = " ".join([out_voc[i] for i in actions]).strip().split(" ")
actions = [a for a in actions if a not in ["<sos>", "<eos>"]]
actions = " ".join(actions)

print(actions)

I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_RIGHT I_TURN_RIGHT I_RUN


# Transformer

Demo en COLAB: [https://colab.research.google.com/drive/1RyAa8wZw0c156xODGotz6kd1AeA2SRLG?usp=sharing](https://colab.research.google.com/drive/1RyAa8wZw0c156xODGotz6kd1AeA2SRLG?usp=sharing)